In [22]:
import pyrealsense2 as rs
import numpy as np
import cv2
import time
import json

# Application Config

In [23]:
EnableColorStream = False
ColorStreamResWidth = 1920
ColorStreamResHeight = 1080
ColorStreamFPS = 30

EnableInfraredStream = True
InfraredStreamResWidth = 1280
InfraredStreamResHeight = 720
InfraredStreamFPS = 30

EnableDepthStream = True
DepthStreamResWidth = 1280
DepthStreamResHeight = 720
DepthStreamFPS = 30

maxDistance = 1000

colorLine = (0, 255, 0)
sizeLine = 3
colorLineP = (0, 255, 255)
sizeLineP = 1

# Application flags and global parameters

In [24]:
capture = False
point = False

paint = False
display = False
paintLTX = -1
paintLTY = -1
paintRDX = -1
paintRDY = -1

img = np.zeros((512,512,3), np.uint8)

click = False
img2 = np.zeros((512,512,3), np.uint8)

# RealSense Camera Config

In [25]:
# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()

if EnableColorStream is True:
    config.enable_stream(
        rs.stream.color, ColorStreamResWidth, ColorStreamResHeight, rs.format.bgr8, ColorStreamFPS)

if EnableInfraredStream is True:
    config.enable_stream(
        rs.stream.infrared, InfraredStreamResWidth, InfraredStreamResHeight, rs.format.bgr8, InfraredStreamFPS)
    
if EnableDepthStream is True:
    config.enable_stream(
        rs.stream.depth, DepthStreamResWidth, DepthStreamResHeight, rs.format.z16, DepthStreamFPS)

# Start streaming
cfg = pipeline.start(config)

# Alignment
if EnableColorStream is True:
    align_to = rs.stream.color
elif EnableInfraredStream is True:
    align_to = rs.stream.infrared

align = rs.align(align_to)
    
# Advanced settings
dev = cfg.get_device()
depth_sensor.set_option(rs.option.visual_preset, 4)
depth_sensor.set_option(rs.option.enable_auto_exposure, 1)

# Get depth scale
scale = depth_sensor.get_depth_scale()
print("depth scale:" + str(scale))
# PointCloud settings
pc = rs.pointcloud()

depth scale:0.0010000000474974513


# colorize depth image

In [26]:
def procColorMap(input_image):
    minNum, maxNum, minLoc, maxLoc = cv2.minMaxLoc(input_image)

    if maxNum > maxDistance:
        maxNum = maxDistance
    
    scaleAlpha = 255 / maxNum
    input_image = cv2.convertScaleAbs(input_image, None, scaleAlpha, 0)
    depth_colormap = cv2.applyColorMap(input_image, cv2.COLORMAP_JET)
    
    return depth_colormap

# screen capture function

In [27]:
def procScreenCapture():  
    global capture
    if capture is True:
        capture = False
        if EnableColorStream is True:
            fileName = "../imageCapture/color_" + time.strftime("%Y-%m-%d_%H%M%S-", time.localtime()) + '.png'
            cv2.imwrite(fileName, color_image, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])

        if EnableInfraredStream is True:
            fileName = "../imageCapture/infrared_" + time.strftime("%Y-%m-%d_%H%M%S-", time.localtime()) + '.png'
            cv2.imwrite(fileName, infrared_image, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])

        if EnableDepthStream is True:
            fileName = "../imageCapture/depth_" + time.strftime("%Y-%m-%d_%H%M%S-", time.localtime()) + '.png'
            cv2.imwrite(fileName, depth_colormap, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])


# Measure Distace Function

In [28]:
# mouse click function
def procMouse(event,x,y,flags,param):
    global paintLTX, paintLTY, paintRDX, paintRDY
    global click
    global img, img2
    
    if event is cv2.EVENT_LBUTTONDOWN:
        paintLTX = x
        paintLTY = y
        click = True
        img2 = img.copy()
        
    elif event is cv2.EVENT_MOUSEMOVE:
        if click is True:
            paintRDX = x
            paintRDY = y

    elif event is cv2.EVENT_LBUTTONUP:
        paintRDX = x
        paintRDY = y
        click = False

In [29]:
# mouse paint function
def procPaint():
    global paint, display, click
    global paintLTX, paintLTY, paintRDX, paintRDY
    global img, img2
    
    if paint is True:
        img = infrared_image
        cv2.namedWindow('Display')
        cv2.setMouseCallback('Display', procMouse)
        cv2.imshow('Display', img)
        display = True
        paint = False
    
    if display is True:
        if click is True:
            img2 = img.copy()
            if paintLTX != -1 and paintRDX != -1:
                cv2.line(img2, (paintLTX,paintLTY), (paintRDX, paintRDY), colorLineP, sizeLineP)
            cv2.imshow('Display', img2)
        else:
            if paintLTX != -1 and paintRDX != -1:
                cv2.line(img, (paintLTX,paintLTY), (paintRDX, paintRDY), colorLine, sizeLine)
                paintLTX = -1
                paintLTY = -1
                paintRDX = -1
                paintRDY = -1

            cv2.imshow('Display', img)

In [30]:
def procPointCloud():
    global point
    
    if point is True:
        point = False
        points = pc.calculate(depth_frame)
        
        if EnableColorStream is True:
            pc.map_to(color_frame)
            
        if EnableInfraredStream is True:
            pc.map_to(infrared_frame)
            
        vertices = points.get_vertices()
        tex_coords = points.get_texture_coordinates()
        
        for i in range(0, points.size()):
            if vertices[i].z:
                print(vertices[i].x)
                print(vertices[i].y)
                print(vertices[i].z)
                break

# Main Loop

In [31]:
while True:
    # Wait for a coherent pair of frames: depth and color
    frames = pipeline.wait_for_frames()
    
    # Aligh frames
    aligned_frames = align.proccess(frames)
    
    if EnableColorStream is True:
        color_frame = aligned_frames.get_color_frame()
        if not color_frame:
            continue
            
    if EnableInfraredStream is True:
        infrared_frame = aligned_frames.first(rs.stream.infrared)
        if not infrared_frame:
            continue
    
    if EnableDepthStream is True:
        depth_frame = aligned_frames.get_depth_frame()
        if not depth_frame:
            continue
    
    # Convert images to numpy arrays
    if EnableColorStream is True:
        color_image = np.asanyarray(color_frame.get_data())
    
    if EnableInfraredStream is True:
        infrared_image = np.asanyarray(infrared_frame.get_data())
    
    if EnableDepthStream is True:
        depth_image = np.asanyarray(depth_frame.get_data())
    
    # colorize depth image
    depth_colormap = procColorMap(depth_image)
   
    # Show images
    if EnableColorStream is True:
        cv2.namedWindow('Color image', 0)
        cv2.imshow('Color image', color_image)
        
    if EnableInfraredStream is True:
        cv2.namedWindow('Infrared image', 0)
        cv2.imshow('Infrared image', infrared_image)
    
    if EnableDepthStream is True:
        cv2.namedWindow('Depth image', 0)
        cv2.imshow('Depth image', depth_colormap)
    
    # Screen capture
    procScreenCapture()
        
    # Read device temperature
    '''
    temp = depth_sensor.get_option(rs.option.projector_temperature)
    print("proj: " + str(temp))
    temp = depth_sensor.get_option(rs.option.asic_temperature)
    print("asic: " + str(temp))
    '''
    
    # Generate point cloud
    procPointCloud()
    
    # Painit function
    procPaint()
        
    # Keyboard command
    getKey = cv2.waitKey(10) & 0xFF
    if getKey is ord('c') or getKey is ord('C'):
        capture = True
    elif getKey is ord('q') or getKey is ord('Q'):
        break
    elif getKey is ord('p') or getKey is ord('P'):
        point = True
    elif getKey is ord('a') or getKey is ord('A'):
        paint = True
    elif getKey is ord('x') or getKey is ord('X'):
        cv2.destroyAllWindows()

In [32]:
# Stop streaming
cv2.destroyAllWindows()
pipeline.stop()